In [1]:
import pandas as pd

path = r"C:\Users\sebastian sossa\Documents\integraIT\projects_integrait\DNDI\data\db3bea42-add2-4185-9a08-d8a8a2622277.xlsx"
df_root = pd.read_excel(path)

In [2]:
df = df_root[df_root['name']=='Demographics']
lista_sujetos = df['Participante'].unique()
df = df[['name', 'Visit', 'activityState', 'Participante', 'Estado del Participante', 'Campo', 'Valor']]

In [3]:
df.Campo.unique()

array(['Birth Month', 'Birth Year', 'Age at consent', 'Gender',
       'Ethnicity', 'Race'], dtype=object)

In [6]:
df_demographic_age = df_root[df_root['name']=='Demographics']
df_demographic_age = df_demographic_age[['Visit','Participante', 'Campo', 'Valor']]
df_demographic_age = df_demographic_age[df_demographic_age['Campo']=='Birth Month']
df_demographic_age = df_demographic_age[['Visit','Participante','Valor']]
df_demographic_age = df_demographic_age.rename(columns={'Participante':'Subject', 'Valor':'Birth Year'})

In [7]:
df_demographic_age

,Visit,Subject,Birth Year
1107,Screening Visit,011002,2
1108,Screening Visit,011003,8
1109,Screening Visit,011004,2
1110,Screening Visit,011008,3
1111,Screening Visit,011005,1
1112,Screening Visit,011006,12
1113,Screening Visit,011007,7
1114,Screening Visit,012009,7
1115,Screening Visit,012010,4
1116,Screening Visit,012011,7


In [7]:
df_visit_date = df_root[df_root['name']=='Date of visit']
df_visit_date = df_visit_date[['Visit','Participante', 'Campo', 'Valor']]
df_visit_date = df_visit_date[df_visit_date['Campo']=='Visit Date']
df_visit_date = df_visit_date[['Visit','Participante','Valor']]
df_visit_date = df_visit_date.rename(columns={'Participante':'Subject'})

In [8]:
prueba = df[['Campo', 'Valor']].T
columns_analize = prueba.iloc[0].unique()
columns_analize

array(['Birth Month', 'Birth Year', 'Age at consent', 'Gender',
       'Ethnicity', 'Race'], dtype=object)

In [11]:
from datetime import datetime
from revision_fechas import revision_fecha
import warnings

warnings.filterwarnings('ignore')

lista_revision = []

# fecha_inicio = datetime.strptime('19-06-2023', "%d-%m-%Y")
# fecha_fin =  datetime.strptime('31-10-2023', "%d-%m-%Y")

for sujeto in lista_sujetos:
    sujeto_principal = df[df['Participante']==sujeto]
    for visita in sujeto_principal.Visit.unique():
        pru_1 = sujeto_principal[sujeto_principal['Visit']==visita]
        pru = pru_1
        pru = pru[['Campo', 'Valor']].T
        new_columns = pru.iloc[0]
        pru = pru[1:].set_axis(new_columns, axis=1)
        pru['Subject'] = sujeto
        pru['Visit'] = visita
        pru['status'] = pru_1['activityState'].unique()
        pru = pru.merge(df_visit_date, on=['Subject', 'Visit'], how='left')
        

        for index, row in pru.iterrows():
            status = row['status']
            if status == 'INCOMPLETE' or status == 'PENDING_QUERIES':
                pass
            else:
                subject = row['Subject']
                visit = row['Visit']
                fecha_visita = row['Valor']

                try:
                    birth_month = row['Birth Month']
                    birth_year = row['Birth Year']
                    age_at_consent =  row['Age at consent']
                    gender = row['Gender']
                    ethnicity = row['Ethnicity']
                    race = row['Race']


                    # if int(birth_year) < 1973 or int(birth_year) > 2005:
                    #     error = [subject, visit, 'The year should be between 1973 and 2005' , birth_year, 'DM0010']
                    #     lista_revision.append(error)
                    # else:
                    #     pass

                    # if int(age_at_consent) < 18 or int(age_at_consent) > 50:
                    #     error = [subject, visit, 'The age must be between 18 and 50 years old' , age_at_consent, 'DM0020']
                    #     lista_revision.append(error)
                    # else:
                    #     pass
    
                    año_visita = row['Valor'].split('-')[2]
                    año_calculado = int(año_visita) - int(birth_year)

                    if age_at_consent >= año_calculado -1 and age_at_consent <= año_calculado + 1:
                        pass
                    else:
                        error = [subject, visit, 'The subject AGE at consent doesnt match the AGE according to the month and year of birth' , age_at_consent, 'DM0030']
                        lista_revision.append(error)
                except:
                    pass

In [12]:
lista_revision

[]